In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from os import listdir
from os.path import isfile, join
import tensorflow as tf
import cv2
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Model
from tensorflow.keras.layers import Input,Activation,Add, MaxPooling2D, MaxPooling1D, Flatten, Dense, Conv1D, Dropout
import gc
from ipynb.fs.full.SIFT_to_Features import SIFT_nparray_to_Features
from tensorflow import keras

In [109]:
#Simplifies adding a layer
def Convolution(input_tensor,filters):

    x = layers.Conv2D(filters=filters,kernel_size=(3, 3),padding = 'same', activation = 'relu')(input_tensor)

    return x

def Convolution1D(input_tensor,filters):

    x = layers.Conv1D(filters=filters,kernel_size=(3),padding = 'same', activation = 'relu')(input_tensor)

    return x

#Simple CNN model
#Conv stands for neural network layers of (x,y). x standing for the input layers and y standing for how many filter layers there are
#maxp is a pooling layer that reduces the number of nodes of the next layer
#flatten changes the previous layer to a 1 dimensional layer
#dense layers have each node connected to every node in the previous layer. In this case, we have 3 dense layers of differing weights representing
#age, gender, and race. While all previous layers have collected features, dense layers calculate based on these collected features.
#They then send their calculations to the final dense layer to determine classification
#Binary Crossentropy = Classifies based on a binary value
#Sparse Categorical Crossentropy = Classifies based on an integer value with more than 2 possible values
def Simplemodel(input_shape):

  inputs = Input((input_shape))

  conv_1= Convolution(inputs,32)

  maxp_1 = MaxPooling2D(pool_size = (2,2)) (conv_1)

  conv_2 = Convolution(maxp_1,64)

  maxp_2 = MaxPooling2D(pool_size = (2, 2)) (conv_2)

  conv_3 = Convolution(maxp_2,64)

  maxp_3 = MaxPooling2D(pool_size = (2, 2)) (conv_3)

  conv_4 = Convolution(maxp_3,64)

  maxp_4 = MaxPooling2D(pool_size = (2, 2)) (conv_4)

  conv_5 = Convolution(maxp_4,64)

  flatten= Flatten() (conv_5)

  dense_1= Dense(64,activation='relu')(flatten)

  output_1= Dense(9,activation="relu",name='age_out')(dense_1)

  model = Model(inputs=[inputs], outputs=[output_1])

  model.compile(loss=["sparse_categorical_crossentropy"], optimizer="Adam",

  metrics=["accuracy"])

  return model

def Simplemodel1D(input_shape):

  inputs = Input((input_shape))

  conv_1= Convolution1D(inputs,32)

  maxp_1 = MaxPooling1D(pool_size = 2) (conv_1)

  conv_2 = Convolution1D(maxp_1,64)

  maxp_2 = MaxPooling1D(pool_size = 2) (conv_2)

  conv_3 = Convolution1D(maxp_2,64)

  maxp_3 = MaxPooling1D(pool_size = 2) (conv_3)

  conv_4 = Convolution1D(maxp_3,64)

  maxp_4 = MaxPooling1D(pool_size = 2) (conv_4)

  conv_5 = Convolution1D(maxp_4,64)

  flatten= Flatten() (conv_5)

  dense_1= Dense(64,activation='relu')(flatten)

  output_1= Dense(9,activation="relu",name='age_out')(dense_1)

  model = Model(inputs=[inputs], outputs=[output_1])

  model.compile(loss=["sparse_categorical_crossentropy"], optimizer="Adam",

  metrics=["accuracy"])

  return model

In [3]:
# Get data and convert for model
data = np.load('../data/raw/UnbalancedRaw/npimagearrays.npy',allow_pickle=True)
filelabels = pd.read_csv('../data/raw/UnbalancedRaw/npimagefilelabels.csv').loc[:,['file','age']]

X_train, X_test, y_train, y_test = train_test_split(data, filelabels, test_size=0.33, random_state=42)

y_testnp = np.asarray(y_test.loc[:,'age'])

y_trainnp = np.asarray(y_train.loc[:,'age'])

In [6]:
# Use SIFT if necessary
X_train_sift,X_test_sift,test = SIFT_nparray_to_Features(X_train[:2380],X_test[:2380])

C:\Users\thanh\Documents\GitHub\Facial-Attribute-Classification-by-ML\Notebooks\SIFT_to_Features.ipynb:67: RuntimeWarning: invalid value encountered in true_divide
  "        for j in bin_assignment:\n",


In [5]:
print(test)

test


In [213]:
X_train_sift.shape

(2380, 200)

In [91]:
X_test_sift.shape

(7823, 200)

In [82]:
#Creating the model
Modelsimple=Simplemodel((200,200,3))
Modelsimple.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 conv2d_20 (Conv2D)          (None, 200, 200, 32)      896       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 100, 100, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 100, 100, 64)      18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 50, 50, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 50, 50, 64)        3692

In [119]:
Modelsift=Simplemodel1D((200,1))
Modelsift.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 200, 1)]          0         
                                                                 
 conv1d_19 (Conv1D)          (None, 200, 32)           128       
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 100, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_20 (Conv1D)          (None, 100, 64)           6208      
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 50, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_21 (Conv1D)          (None, 50, 64)            1235

In [83]:
#Training the model
SimpleHistory=Modelsimple.fit(X_train,y_trainnp - 1,validation_data=(X_test,y_testnp - 1),epochs=10)

Epoch 1/10
259/497 [==============>...............] - ETA: 2:22 - loss: 7.4000 - accuracy: 0.4137

KeyboardInterrupt: 

In [112]:
#Training the model
SimpleSIFTHistory=Modelsift.fit(X_train_sift,y_trainnp - 1,validation_data=(X_test_sift,y_testnp - 1),epochs=10)

Epoch 1/10
497/497 [==============================] - 4s 8ms/step - loss: nan - accuracy: 0.0701 - val_loss: nan - val_accuracy: 0.0698
Epoch 2/10
497/497 [==============================] - 4s 8ms/step - loss: nan - accuracy: 0.0667 - val_loss: nan - val_accuracy: 0.0698
Epoch 3/10
497/497 [==============================] - 4s 7ms/step - loss: nan - accuracy: 0.0667 - val_loss: nan - val_accuracy: 0.0698
Epoch 4/10
497/497 [==============================] - 4s 8ms/step - loss: nan - accuracy: 0.0667 - val_loss: nan - val_accuracy: 0.0698
Epoch 5/10
497/497 [==============================] - 4s 8ms/step - loss: nan - accuracy: 0.0667 - val_loss: nan - val_accuracy: 0.0698
Epoch 6/10
497/497 [==============================] - 4s 8ms/step - loss: nan - accuracy: 0.0667 - val_loss: nan - val_accuracy: 0.0698
Epoch 7/10
497/497 [==============================] - 4s 8ms/step - loss: nan - accuracy: 0.0667 - val_loss: nan - val_accuracy: 0.0698
Epoch 8/10
497/497 [============================

In [115]:
tf.convert_to_tensor(X_test_sift)

<tf.Tensor: shape=(7823, 200), dtype=float64, numpy=
array([[0.0804518 , 0.0402259 , 0.0804518 , ..., 0.        , 0.0402259 ,
        0.0402259 ],
       [0.05802589, 0.23210354, 0.05802589, ..., 0.05802589, 0.05802589,
        0.05802589],
       [0.05577114, 0.02788557, 0.02788557, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.03940552, 0.07881104, 0.03940552, ..., 0.        , 0.03940552,
        0.03940552],
       [0.25175441, 0.        , 0.25175441, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.05463584, ..., 0.        , 0.        ,
        0.10927167]])>

In [149]:

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 0, 10, 32
	model = keras.Sequential()
	model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(200,1)))
	model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
	model.add(MaxPooling1D(pool_size=2))
	model.add(Flatten())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(9, activation='softmax'))
	model.compile(loss="sparse_categorical_crossentropy", optimizer='Adamax', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy)
	return accuracy

In [155]:
newytrain = y_trainnp - 1
newytest = y_testnp - 1

In [208]:
evaluate_model(np.nan_to_num(X_train_sift)[:],newytrain[:],np.nan_to_num(X_test_sift)[:1000],newytest[:1000])

Epoch 1/10
497/497 [==============================] - 6s 11ms/step - loss: 1.6185 - accuracy: 0.4464
Epoch 2/10
497/497 [==============================] - 6s 11ms/step - loss: 1.4704 - accuracy: 0.4858
Epoch 3/10
497/497 [==============================] - 6s 12ms/step - loss: 1.4338 - accuracy: 0.4934
Epoch 4/10
497/497 [==============================] - 6s 12ms/step - loss: 1.4092 - accuracy: 0.5004
Epoch 5/10
497/497 [==============================] - 6s 12ms/step - loss: 1.3901 - accuracy: 0.5020
Epoch 6/10
497/497 [==============================] - 6s 12ms/step - loss: 1.3745 - accuracy: 0.5048
Epoch 7/10
497/497 [==============================] - 6s 11ms/step - loss: 1.3552 - accuracy: 0.5111
Epoch 8/10
497/497 [==============================] - 6s 12ms/step - loss: 1.3379 - accuracy: 0.5136
Epoch 9/10
497/497 [==============================] - 6s 11ms/step - loss: 1.3173 - accuracy: 0.5236
Epoch 10/10
32/32 [==============================] - 0s 3ms/step - loss: 1.4470 - accuracy:

0.5109999775886536

In [137]:
y_testnp - 1

array([1, 1, 8, ..., 4, 0, 2], dtype=int64)

In [142]:
y_trainnp - 1

array([4, 5, 3, ..., 4, 6, 3], dtype=int64)

In [152]:
Boolarray = X_train_sift == None

In [7]:
X_train_sift[2372]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])